In [2]:
import csv 
import requests
import os
import pandas as pd
from openai import OpenAI
from bs4 import BeautifulSoup
from typing import Dict, List
from urllib.parse import urlparse
import tkinter as tk
from tkinter import filedialog

In [3]:
client = OpenAI(
    api_key = 'sk-svcacct-XNnYiDVzrJ3Tb8q5SSE_wy2n6L6AAks3onXlCHR2iifEOomzqilw3-cXHG6ZD6aD4rqcBL3R8T3BlbkFJUwgZhxvBp9XVBnCFYQvUFQtNzG--VDzLNKkudsq4YP4cNveJm1r9stlPt57-drV-cKUPGl5-wA',
)

In [ ]:
csv_url = input("Enter the URL of the CSV file: ")
df = pd.read_csv(csv_url)
df.head()

In [5]:
def scrape_websites(Website):
    
    try:
        response = requests.get(Website, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')

        data = {
            'url': Website,
            'title': soup.title.string if soup.title else 'No title found',
            'meta_description': soup.find('meta', attrs={'name': 'description'})['content'] if soup.find('meta', attrs={'name': 'description'}) else 'No meta description found',
            'headings': [h.text for h in soup.find_all(['h1', 'h2', 'h3'])[:5]],
            'paragraphs': [p.text for p in soup.find_all('p')[:20]]
        }

        return data
    
    except Exception as e:
        return "error"

In [ ]:
df['website_data'] = df['Website'].apply(scrape_websites)
df.head()

In [7]:
def gpt_prompt_generate_perssonalized_from_website(row):
    
    website_data = row['website_data']
    company_name = row['Company']
        
    prompt = f""" Write an introductory line to an email that sounds friendly and personal. Avoid formal or exaggerated expressions, 
    and aim for a tone that is casual, friendly, and sounds like a real person speaking. 
    Look at the following company’s website description and use it to share a positive detail about their company’s business: 
    {website_data}. Start the line with ‘I was on your site and saw {company_name}'  and continue by sharing something they value, 
    prioritize, or are aiming to accomplish that relates to Litespace. 
    Do not include any quotation marks and write just one example, do not make a list of examples.
    Keep the sentence below 30 words and do not quote their description directly, meaning change up wording
    """
    
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that writes personalized outbound sales emails for Litespace who is a startup digitizing human resources and driving performance through improving employee experience."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=200,
            n=1,
            stop=None,
            temperature=1.0
            )

        generated_email = response.choices[0].message.content
        return generated_email

    except Exception as e:
         return f"Error: {e}"

In [ ]:
df['Message'] = df.apply(gpt_prompt_generate_perssonalized_from_website, axis=1)
df.head()

In [ ]:
root = tk.Tk()
root.withdraw()
save_folder = filedialog.askdirectory(title="Select a folder to save the file")

if save_folder:
    file_path = os.path.join(save_folder, "result.csv")
    df.to_csv(file_path, encoding='utf-8-sig', index=False)
    print(f"File saved to: {file_path}")
else:
    print("No folder selected. File not saved.")